In [1]:
# enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from askharrison.arxiv_search import expand_arxiv_query, run_multi_arixv_queries

In [3]:
problem_statement = """RAG(retrieval augmented generation) sometimes the sources retrieved are relevant but not enough to answer the question user ask. How do research approach this case"""

In [4]:
problem_statement = """reduce hallucination in RAG(retrieval augmented generation)"""

In [5]:
search_queries = expand_arxiv_query(problem_statement)

In [6]:
search_queries

['RAG model hallucination reduction techniques',
 'Reducing hallucination in Retrieval Augmented Generation',
 'Methods to minimize hallucination in RAG models',
 'RAG model accuracy improvement strategies',
 'Techniques for efficient and accurate RAG models',
 'Overcoming hallucination in RAG models',
 'Mitigating hallucination in Retrieval Augmented Generation',
 'Accuracy enhancement in RAG models',
 'Approaches to reduce hallucination in RAG',
 'Solutions for hallucination in Retrieval Augmented Generation']

In [7]:
arxiv_query_results = run_multi_arixv_queries(search_queries)

100%|██████████| 10/10 [00:22<00:00,  2.25s/it]


In [8]:
len(arxiv_query_results.items())

10

In [9]:
# flatten arxiv_query_results
all_results = []
for query in arxiv_query_results:
    for result in arxiv_query_results[query]:
        all_results.append(result)

# make arxiv query results a dataframe and create a new dataframe with only unique entry_id
import pandas as pd

arixv_result_df = pd.DataFrame(all_results)
unique_arixv_result_df = arixv_result_df.drop_duplicates(subset='entry_id')

In [10]:
arixv_result_df.shape, unique_arixv_result_df.shape

((200, 13), (116, 13))

In [11]:
arixv_result_df.groupby('entry_id').count().sort_values('title', ascending=False).head(10)

,authors,categories,comment,doi,journal_ref,links,pdf_url,primary_category,published,summary,title,updated
entry_id,,,,,,,,,,,,
http://arxiv.org/abs/2410.11414v2,7,7,7,0,0,7,7,7,7,7,7,7
http://arxiv.org/abs/2407.12325v1,7,7,0,0,0,7,7,7,7,7,7,7
http://arxiv.org/abs/2411.12759v1,6,6,0,0,0,6,6,6,6,6,6,6
http://arxiv.org/abs/2401.00396v2,5,5,0,0,0,5,5,5,5,5,5,5
http://arxiv.org/abs/2408.15533v2,5,5,0,0,0,5,5,5,5,5,5,5
http://arxiv.org/abs/2412.04235v1,4,4,0,0,0,4,4,4,4,4,4,4
http://arxiv.org/abs/2410.13085v1,4,4,0,0,0,4,4,4,4,4,4,4
http://arxiv.org/abs/2410.18251v1,3,3,3,0,0,3,3,3,3,3,3,3
http://arxiv.org/abs/2408.00555v1,3,3,0,0,0,3,3,3,3,3,3,3


In [12]:
from askharrison.prompts.content_curation import create_arxiv_filtering_prompt

In [13]:
help(create_arxiv_filtering_prompt)

Help on function create_arxiv_filtering_prompt in module askharrison.prompts.content_curation:

create_arxiv_filtering_prompt(problem_statement: str, doc_abstract: str)



In [14]:
# create a prompt for each arxiv entry
arxiv_reranking_prompts = [create_arxiv_filtering_prompt(problem_statement, 
                                         record['title']+"\n"+record['summary']) for record in unique_arixv_result_df.to_dict(orient='records')]

In [15]:
from askharrison.llm_models import parallel_llm_processor, process_question, safe_eval, extract_python_code

In [16]:
reranking_llm_response = parallel_llm_processor(arxiv_reranking_prompts, llm_function=process_question, 
                                                max_workers=8)

Processing prompts: 100%|██████████| 116/116 [00:57<00:00,  2.03it/s]


In [17]:
llm_responses_results = [safe_eval(extract_python_code(response)) for response in reranking_llm_response]
# filter out empty responses

unique_arixv_result_df.shape, len(llm_responses_results)

((116, 13), 116)

In [18]:
# extract reasoning, is_direct, 'is_relevant' from llm_responses_results if it is not empty, and add to unique_arixv_result_df
unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_relevant'] = [response['is_relevant'] if response else None for response in llm_responses_results]


C:\Users\alist\AppData\Local\Temp\ipykernel_103088\658567496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykernel_103088\658567496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykerne

In [19]:
# increase max column width in pandas
pd.set_option('display.max_colwidth', None)

In [22]:
unique_arixv_result_df.query('is_direct == True').shape

(41, 16)

In [ ]:
unique_arixv_result_df.query('is_direct == True')\
    [['title','entry_id', 'summary', 'is_relevant']]

,title,entry_id,summary,is_relevant
0,Addressing Hallucinations with RAG and NMISS in Italian Healthcare LLM Chatbots,http://arxiv.org/abs/2412.04235v1,"I combine detection and mitigation techniques to addresses hallucinations in\nLarge Language Models (LLMs). Mitigation is achieved in a question-answering\nRetrieval-Augmented Generation (RAG) framework while detection is obtained by\nintroducing the Negative Missing Information Scoring System (NMISS), which\naccounts for contextual relevance in responses. While RAG mitigates\nhallucinations by grounding answers in external data, NMISS refines the\nevaluation by identifying cases where traditional metrics incorrectly flag\ncontextually accurate responses as hallucinations. I use Italian health news\narticles as context to evaluate LLM performance. Results show that Gemma2 and\nGPT-4 outperform the other models, with GPT-4 producing answers closely aligned\nwith reference responses. Mid-tier models, such as Llama2, Llama3, and Mistral\nbenefit significantly from NMISS, highlighting their ability to provide richer\ncontextual information. This combined approach offers new insights into the\nreduction and more accurate assessment of hallucinations in LLMs, with\napplications in real-world healthcare tasks and other domains.",5
1,Luna: An Evaluation Foundation Model to Catch Language Model Hallucinations with High Accuracy and Low Cost,http://arxiv.org/abs/2406.00975v2,"Retriever Augmented Generation (RAG) systems have become pivotal in enhancing\nthe capabilities of language models by incorporating external knowledge\nretrieval mechanisms. However, a significant challenge in deploying these\nsystems in industry applications is the detection and mitigation of\nhallucinations: instances where the model generates information that is not\ngrounded in the retrieved context. Addressing this issue is crucial for\nensuring the reliability and accuracy of responses generated by large language\nmodels (LLMs) in diverse industry settings. Current hallucination detection\ntechniques fail to deliver accuracy, low latency, and low cost simultaneously.\nWe introduce Luna: a DeBERTA-large (440M) encoder, finetuned for hallucination\ndetection in RAG settings. We demonstrate that Luna outperforms GPT-3.5 and\ncommercial evaluation frameworks on the hallucination detection task, with 97%\nand 91% reduction in cost and latency, respectively. Luna is lightweight and\ngeneralizes across multiple industry verticals and out-of-domain data, making\nit an ideal candidate for industry LLM applications.",5
3,LRP4RAG: Detecting Hallucinations in Retrieval-Augmented Generation via Layer-wise Relevance Propagation,http://arxiv.org/abs/2408.15533v2,"Retrieval-Augmented Generation (RAG) has become a primary technique for\nmitigating hallucinations in large language models (LLMs). However, incomplete\nknowledge extraction and insufficient understanding can still mislead LLMs to\nproduce irrelevant or even contradictory responses, which means hallucinations\npersist in RAG. In this paper, we propose LRP4RAG, a method based on the\nLayer-wise Relevance Propagation (LRP) algorithm for detecting hallucinations\nin RAG. Specifically, we first utilize LRP to compute the relevance between the\ninput and output of the RAG generator. We then apply further extraction and\nresampling to the relevance matrix. The processed relevance data are input into\nmultiple classifiers to determine whether the output contains hallucinations.\nTo the best of our knowledge, this is the first time that LRP has been used for\ndetecting RAG hallucinations, and extensive experiments demonstrate that\nLRP4RAG outperforms existing baselines.",5
5,AlzheimerRAG: Multimodal Retrieval Augmented Generation for PubMed articles,http://arxiv.org/abs/2412.16701v1,"Recent advancements in generative AI have flourished the development of\nhighly adept Large Language Models (LLMs) that integrate diverse data types to\nempower decision-making. Among these, 